In [0]:
%sql
-- Celda 1: Configurar el contexto
USE CATALOG sesion_5;
USE SCHEMA gold;

In [0]:
%sql
-- Celda 2: Crear la tabla de características del cliente (RFM)
-- Esta tabla será la entrada para nuestro modelo de K-Means.
CREATE OR REPLACE TABLE customer_features_gold AS
WITH
  -- Paso 1: Calcular la fecha de la última compra en todo el dataset para tener un punto de referencia consistente para la recencia.
  max_order_date AS (
    SELECT MAX(fecha_pedido) AS max_date FROM silver.fact_pedidos
  ),
  
  -- Paso 2: Calcular las métricas base de RFM por cliente único.
  rfm_metrics AS (
    SELECT
      c.customer_unique_id,
      -- Recencia: ¿Qué tan recientemente compró el cliente? (Menor es mejor)
      DATEDIFF((SELECT max_date FROM max_order_date), MAX(fo.fecha_pedido)) AS recency,
      -- Frecuencia: ¿Con qué frecuencia compra el cliente?
      COUNT(DISTINCT fo.order_id) AS frequency,
      -- Monetario: ¿Cuánto gasta el cliente?
      SUM(fo.valor_pago) AS monetary
    FROM
      silver.fact_pedidos fo
    JOIN
      silver.dim_clientes_sql c ON fo.customer_id = c.customer_id
    WHERE fo.order_status = 'delivered' -- Solo se consideran pedidos completados.
    GROUP BY
      c.customer_unique_id
  )

-- Paso 3: Seleccionar las características finales para la tabla Gold.
SELECT
  customer_unique_id,
  recency,
  frequency,
  monetary
FROM rfm_metrics;


In [0]:
# Taller de Machine Learning: Segmentación de Clientes con Sklearn y MLflow

# ==============================================================================
# Celda 1: Importar las librerías necesarias
# ==============================================================================
# MLflow para el seguimiento de experimentos.
import mlflow

# Pandas para la manipulación de datos, que es el formato que prefiere scikit-learn.
import pandas as pd

# Scikit-learn para el preprocesamiento y el modelo de clustering.
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Se necesita la sesión de Spark para obtener el nombre de usuario actual.
from pyspark.sql import SparkSession

In [0]:
# ==============================================================================
# Celda 2: Cargar los datos de características desde la capa Gold
# ==============================================================================
# Se cargan los datos de la tabla Gold que se preparó en el notebook anterior.
# Se convierten a un DataFrame de Pandas para poder usarlos con scikit-learn.
print("Cargando datos desde sesion_5.gold.customer_features_gold...")
features_df = spark.table("sesion_5.gold.customer_features_gold").toPandas()

# Se establece el 'customer_unique_id' como el índice para facilitar la unión de los resultados más tarde.
features_df.set_index("customer_unique_id", inplace=True)

print(f"Se cargaron {len(features_df)} registros de clientes.")

In [0]:
# ==============================================================================
# Celda 3: Preparar los datos para el modelo
# ==============================================================================
# K-Means es sensible a la escala de las características. Por ejemplo, 'monetary'
# tendrá valores mucho más grandes que 'frequency'. El escalado asegura que
# todas las características contribuyan por igual al cálculo de la distancia.

print("Escalando características (Recencia, Frecuencia, Monetario)...")
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_df)


In [0]:
# ==============================================================================
# Celda 4: Configurar y ejecutar el experimento de MLflow
# ==============================================================================
# Se obtiene el nombre de usuario actual de forma programática para crear una ruta de experimento única y válida.
# Esto soluciona el error 'RESOURCE_DOES_NOT_EXIST' al asegurar que MLflow siempre tenga una ruta válida para guardar el experimento.
user_email = spark.sql("SELECT current_user()").first()[0]
experiment_path = f"/Users/{user_email}/customer_segmentation_sklearn"
mlflow.set_experiment(experiment_path)
print(f"Experimento de MLflow configurado en: {experiment_path}")

In [0]:
# Se inicia una nueva ejecución (run) dentro del experimento.
# Todo lo que se registre (parámetros, métricas, modelos) quedará asociado a esta ejecución.
with mlflow.start_run() as run:
    # --- Habilitar el autologging para scikit-learn ---
    # Esta es la magia de MLflow. Registrará automáticamente los parámetros del modelo,
    # las métricas de entrenamiento y el modelo final sin necesidad de llamadas explícitas.
    mlflow.autolog()

    # NOTA SOBRE EL WARNING: El warning "Training metrics will not be recorded because training labels were not specified"
    # es esperado. K-Means es un algoritmo de clustering (no supervisado) y no tiene "etiquetas" (labels) de entrenamiento.
    # MLflow autolog() está optimizado para modelos supervisados, pero aún así registrará los parámetros y el modelo correctamente.

    # --- Definir y entrenar el modelo ---
    # A diferencia de la regresión, el clustering es un aprendizaje no supervisado.
    # No hay un "y_test" para comparar. El objetivo es encontrar patrones en los datos,
    # por lo que se entrena con todo el conjunto de datos.
    num_clusters = 4 # Se puede experimentar con este valor.
    
    print(f"Entrenando modelo K-Means con k={num_clusters}...")
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    kmeans.fit(features_scaled)

    # --- Evaluar el modelo ---
    # Aunque autolog() podría registrar algunas métricas, la puntuación de silueta
    # es una métrica de evaluación clave para clustering que se debe calcular explícitamente.
    # Mide qué tan similares son los puntos dentro de un clúster en comparación con otros clústeres.
    # Un valor más cercano a 1 es mejor.
    silhouette = silhouette_score(features_scaled, kmeans.labels_)
    mlflow.log_metric("silhouette_score", silhouette)
    
    print(f"Modelo entrenado. Puntuación de Silueta: {silhouette:.3f}")
    
    # Se guardan las predicciones (los segmentos de clientes) en el DataFrame original.
    features_df['segment'] = kmeans.labels_

    run_id = run.info.run_id
    print(f"Ejecución de MLflow completada. ID de la ejecución: {run_id}")

In [0]:
# ==============================================================================
# Celda 5: Guardar los resultados de la segmentación en una tabla Gold
# ==============================================================================
# Finalmente, se guardan los resultados para que el equipo de negocio pueda utilizarlos.
# Se convierte el DataFrame de Pandas de nuevo a un DataFrame de Spark para guardarlo como tabla Delta.
print("Guardando los segmentos de clientes en la tabla gold...")
results_spark_df = spark.createDataFrame(features_df.reset_index())

results_spark_df.select("customer_unique_id", "segment") \
    .write.format("delta").mode("overwrite") \
    .saveAsTable("sesion_5.gold.customer_segments_gold")

print("¡Proceso completado! La tabla 'customer_segments_gold' ha sido creada.")


In [0]:
# ==============================================================================
# Celda 6: Revisión de los resultados
# ==============================================================================
# Se muestra una vista previa de los segmentos asignados.
display(results_spark_df)
